# Workflow:

1. Problem description, data characteristics, visualisation, distributions, etc.

2. Preprocessing:
   - missing data (delete, leave, imputation + explaination);
   - normalization/standarization (type of transformation + description, transformation categorical data);
   - separation - test and training sets (training/test, cross-validation, proportions of subsets);

3. Building and training neural network:
    - layers, neurons quantity; 
    - activation functions (+description, last layer should transform to binary output);
    - optimalisation method (+description, which one returns best results);
    - loss function and metrics (+description);
    - model fitting (epochs number, batch size, model evaluation (training result));
    - test set based classification (evaluation methods of binary classifier, including data type (should include just and only most desriptive metrics

4. Additional methods not presented in labs/lectures.

5. Present different parameters values, to decide, which ones caused accuracy and classification improvement.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy as sp
import keras

Using TensorFlow backend.


# Problem description

...

# Reading data using Pandas

In [62]:
data = pd.read_csv("logreg.txt", sep = ";", dtype={'genotype': np.object})

# Data characteristics

In [63]:
# data.head()
# data.shape
# print(data.dtypes)

# data.BEFORE1.value_counts()
# data.BEFORE2.value_counts()
# data.BEFORE3.value_counts()

# data.BEHIND1.value_counts()
# data.BEHIND2.value_counts()
# data.BEHIND3.value_counts()

# descriptions_dataframe = data.describe(include='all')
# nulls = data.isnull().sum().to_frame(name='NA').T 
# summary_of_nulls = pd.concat([descriptions_dataframe, nulls])
# summary_of_nulls

# data.iloc[:,6:].apply(pd.value_counts)

# Data visualisation

In [64]:
# data.hist()

# Distributions

# Preprocessing

In [ ]:
tmp['BEFORE'] = data.BEFORE1 + data.BEFORE2 + data.BEFORE3
tmp['BEHIND'] = data.BEHIND1 + data.BEHIND2 + data.BEHIND3
data = data.drop(columns=['BEFORE1', 'BEFORE2', 'BEFORE3', 'BEHIND1', 'BEHIND2', 'BEHIND3'])

data.head()
tmp.head()

dummies = pd.get_dummies(tmp)

data = pd.concat([data, dummies])
data.head()

C:\Users\Acer\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
